# 02 — PyTorch Implementation (No Autograd)

## 1. Imports & Deterministic Seeds

In [1]:
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

## 2. Synthetic Dataset (same as Notebook 01)

In [2]:
# X shape: (N, 2), y shape: (N,)
# Use the same seed so samples match NumPy network
N = 200
X_np = np.random.uniform(-1, 1, size=(N, 2))
y_np = (X_np[:, 0] * X_np[:, 1] < 0).astype(np.float64)

#### Convert to PyTorch:

In [3]:
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32)

## 3. NumPy Reference Forward Pass (from Notebook 01)
To ensure numerical parity between the NumPy and PyTorch implementations,
we replicate the minimal forward-pass functions from Notebook 01. These
are used for direct comparison in Section 8.

In [4]:
# NumPy ReLU
def relu_np(x):
    return np.maximum(0, x)

# NumPy forward pass from Notebook 01
def forward_single(x, W1, b1, W2, b2):
    a1 = W1 @ x + b1            # (h,)
    h  = relu_np(a1)            # (h,)
    f  = W2 @ h + b2            # scalar
    return a1, h, float(f.item())

## 4 .NumPy Model Parameters (for comparison)

Notebook 02 needs standalone NumPy parameters to reproduce the exact
forward pass used in Notebook 01. These are synchronized with the
PyTorch parameters in Section 5 so both implementations produce
identical outputs.

In [5]:
# === 4.1 NumPy Parameters (copied from Notebook 01) ===

d, h = 2, 4

# Initialize NumPy parameters exactly like Notebook 01
W1 = np.random.randn(h, d)
b1 = np.random.randn(h)

W2 = np.random.randn(1, h)
b2 = np.random.randn(1)

## 5. Model Parameters in PyTorch (No Autograd)

In [6]:
# Create PyTorch parameters (initial values do NOT matter yet)
W1_t = torch.randn(h, d, dtype=torch.float32)
b1_t = torch.randn(h, dtype=torch.float32)
W2_t = torch.randn(1, h, dtype=torch.float32)
b2_t = torch.randn(1, dtype=torch.float32)

# Disable autograd (for this notebook)
for t in [W1_t, b1_t, W2_t, b2_t]:
    t.requires_grad = False

# --- Sync PyTorch parameters with NumPy parameters ---
W1_t.data = torch.tensor(W1, dtype=torch.float32)
b1_t.data = torch.tensor(b1, dtype=torch.float32)
W2_t.data = torch.tensor(W2, dtype=torch.float32)
b2_t.data = torch.tensor(b2, dtype=torch.float32)

## 5. Activation Function
#### Match NumPy ReLU:

In [7]:
def relu_t(x):
    return torch.clamp(x, min=0.0)

## 6. Forward Pass (PyTorch)

In [8]:
def forward_torch(x, W1, b1, W2, b2):
    a1 = x @ W1.T + b1          # (h,)
    h  = relu_t(a1)             # (h,)
    f  = W2 @ h + b2            # (1,)
    return a1, h, f.squeeze()   # scalar

## 7. Loss Function
#### Match NumPy MSE:

In [9]:
def mse_loss_t(f, y):
    return (f - y)**2

## 8. Numerical Consistency Test (NumPy vs Torch)
We compare the NumPy output from Notebook 01 with the PyTorch output here.

Pick a single sample:

In [10]:
i = 0
x_i_np = X_np[i]
y_i_np = y_np[i]

# Forward using NumPy functions (from Notebook 01)
a1_np, h_np, f_np = forward_single(x_i_np, W1, b1, W2, b2)

# Forward using PyTorch
x_i_t = X[i]
a1_t, h_t, f_t = forward_torch(x_i_t, W1_t, b1_t, W2_t, b2_t)

In [11]:
print("NumPy output f =", f_np)
print("Torch output f =", float(f_t))
print("Difference =", float(abs(f_np - f_t)))

NumPy output f = -1.2906031334882266
Torch output f = -1.2906031608581543
Difference = 0.0


## 9. Conclusion
This notebook recreated the NumPy MLP from Notebook 01 using PyTorch tensors with requires_grad=False. After synchronizing parameters, both implementations produced numerically identical outputs, differing only by negligible float32 precision noise. This confirms that the PyTorch forward pass matches the analytical NumPy model exactly.

With functional parity established, we are now ready to enable PyTorch autograd, compare automatic gradients to our manual backprop, and move toward a full training loop.